In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Resources: 
* https://www.analyticsvidhya.com/blog/2021/09/adaboost-algorithm-a-complete-guide-for-beginners/
* sklearn documentation
* Multi class adaboost Ji Zhu
* AdaBoost-CNN Aboozar

### Imports

In [3]:
!python --version

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Python 3.7.12


In [ ]:
!pip install dlib

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 29.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done

In [2]:
import matplotlib.pyplot as plt

import numpy as np
import cv2

import PIL.Image as Image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.optimizers.experimental import RMSprop, Adam
import gc
import random
import math
import keras.backend as K
from sklearn.model_selection import train_test_split

#import libraries
import keras
from keras.models import Sequential , Model
from keras.applications.vgg16 import VGG16
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Input, Add, Activation, ZeroPadding2D, BatchNormalization, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D , Dense, Conv2D, MaxPool2D , Flatten , concatenate ,DepthwiseConv2D, GlobalAveragePooling2D , ReLU
import pandas as pd
import matplotlib.pyplot as plt
import os
#import dlib

tf.keras.utils.set_random_seed(42)

In [6]:
tf. __version__

'2.9.2'

In [40]:
def ReSize(image, size):
    resizedimg=cv2.resize(image,(size,size))
    return resizedimg 

def normalize(resimg):
    img=cv2.cvtColor(resimg,cv2.COLOR_BGR2RGB)
    img_normalized = cv2.normalize(img, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    img1=cv2.cvtColor(img_normalized,cv2.COLOR_BGR2RGB)
    return img1    

def gray(normimg):
    img=cv2.cvtColor(normimg, cv2.COLOR_BGR2GRAY)
    grey=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    return grey

def Brightnessandcontrast(normimg):
    Bright = 20
    Contrast = 1
    img=cv2.cvtColor(normimg,cv2.COLOR_BGR2RGB)
    adjusted = cv2.convertScaleAbs(img, alpha=Contrast, beta=Bright)
    rgbadjusted=cv2.cvtColor(adjusted,cv2.COLOR_BGR2RGB)
    return rgbadjusted

def mean_noise_reduction(normimg):
    img=cv2.cvtColor(normimg,cv2.COLOR_BGR2RGB)
    filteredimg = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)
    rgbfilteredimg=cv2.cvtColor(filteredimg,cv2.COLOR_BGR2RGB)
    return rgbfilteredimg

def three_noise_reduction(normimg):
    img=cv2.cvtColor(normimg,cv2.COLOR_BGR2RGB)
    filteredimg =cv2.GaussianBlur(img,(5,5),0)
    #filteredimg=cv2.blur(img,(5,5))
    #filteredimg=cv2.medianBlur(img,5)
    rgbfilteredimg=cv2.cvtColor(filteredimg,cv2.COLOR_BGR2RGB)
    return rgbfilteredimg

def face_detection(samdata):
    
    detector = dlib.get_frontal_face_detector()
    greyimg=cv2.cvtColor(samdata, cv2.COLOR_BGR2GRAY)
    detectedimg = detector(greyimg)  
    if len(detectedimg) == 0:
        samdata
    else:
        for face in detectedimg:
            x1 = face.left()
            y1 = face.top()
            x2 = face.right()
            y2 = face.bottom()

            img=cv2.rectangle(samdata, (x1, y1), (x2, y2), (0, 0, 0), 2)
            img_height, img_width, c = img.shape
            cropped = img[max(0, face.top()): min(face.bottom(), img_height),max(0, face.left()): min(face.right(), img_width)]

    return cropped

### Read Data

In [6]:
def read_data(list, photo_size):  
    data = []
    for i in range(len(list)):
        for filename in os.listdir(list[i]):
            img  = cv2.imread(list[i] + filename)
            #img = face_detection(img)
            #img = gray(img)
            #img = Brightnessandcontrast(img)
            #img = mean_noise_reduction(img)
            #img = three_noise_reduction(img)
            img = ReSize(img, photo_size)
            img = normalize(img)
            data.append([img, [i%2, abs(i%2 - 1)] ])
        
    
    random.shuffle(data)
    x = []
    y = []
    for i in range(len(data)):
        x.append(data[i][0])
        y.append(data[i][1])
    
    x = np.array(x)
    y = np.array(y
    return data

### Data split

In [7]:
ta = "/kaggle/input/autistic-children-facial-data-set/train/autistic/"
tn = "/kaggle/input/autistic-children-facial-data-set/train/non_autistic/"
testa = "/kaggle/input/autistic-children-facial-data-set/test/autistic/"
testn = "/kaggle/input/autistic-children-facial-data-set/test/non_autistic/"
va = "/kaggle/input/autistic-children-facial-data-set/valid/autistic/"
vn = "/kaggle/input/autistic-children-facial-data-set/valid/non_autistic/"

lis = [tn, ta , testn, testa, vn, va]


x, y = read_data(lis , 224)
print(x.shape)
print(y.shape)

del ta , tn , testa, testn, va, vn, lis

gc.collect()

0

In [28]:
y.sum(axis = 0)

array([1468, 1468])

In [29]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.85, test_size = 0.15, stratify = y)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, train_size = 0.9, test_size = 0.1, stratify = y_train)
del x, y
gc.collect()

46

In [30]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_valid.shape)
print(y_valid.shape)

(2245, 224, 224, 3)
(2245, 2)
(441, 224, 224, 3)
(441, 2)
(250, 224, 224, 3)
(250, 2)


### Xception

In [18]:
def make_model():
    xception = tf.keras.applications.Xception(input_shape=(224, 224,3), include_top=False, weights="imagenet" )
    
    model = tf.keras.models.Sequential([
    xception,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, "relu"),
    tf.keras.layers.Dense(64, "relu"),
    tf.keras.layers.Dense(2, "softmax")
    ])
      
    return model


### MoibleNet

In [ ]:
def depth_block(x, strides):
    x = DepthwiseConv2D(3,strides=strides,padding='same',  use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x
def single_conv_block(x,filters):
    x = Conv2D(filters, 1,use_bias=False)(x)
    x= BatchNormalization()(x)
    x = ReLU()(x)
    return x
def combo_layer(x,filters,strides):
    x = depth_block(x,strides)
    x = single_conv_block(x, filters)
    return x
def MobileNet(input_shape=(224,224,3),n_classes = 2):
    input =layers.Input( input_shape)
    x = Conv2D(32,3,strides=(2,2),padding = 'same', use_bias=False) (input)
    x =  BatchNormalization()(x)
    x = ReLU()(x)
    x = combo_layer(x,64, strides=(1,1))
    x = combo_layer(x,128,strides=(2,2))
    x = combo_layer(x,128,strides=(1,1))
    x = combo_layer(x,256,strides=(2,2))
    x = combo_layer(x,256,strides=(1,1))
    x = combo_layer(x,512,strides=(2,2))
    for _ in range(5):
        x = combo_layer(x,512,strides=(1,1))
    x = combo_layer(x,1024,strides=(2,2))
    x = combo_layer(x,1024,strides=(1,1))
    x = GlobalAveragePooling2D()(x)
    output = Dense(n_classes,activation='softmax')(x)
    model = Model(input, output)
    return model

# SAMME Algorithm

In [ ]:
def samme_evalution(x,y,n_e, estimators_alphas, estimators):
    predicted = []
    for i in range(n_e):
        predicted.append(estimators[i].predict(x))
        gc.collect()
    _m5ra = []   
    for i in range(n_e):
        _m5ra.append([estimators_alphas[i] * j for j in predicted[i].round()])
    _m5ra = np.array(_m5ra).sum(axis = 0)

    acc = 0
    for i in range(len(x)):
       if (np.argmax(_m5ra[i])) == (np.argmax(y[i])):
            acc += 1 
    return acc / y.shape[0]

In [ ]:
estimators = []  # list of models
estimators_weights = [] # list of CNN weights of each model
estimators_alphas = [] # 
sample_weights = [] # list of sample weights of each model
n_e = 6 # number of models
gc.collect()

for i in range(0, n_e):
    gc.collect()
    # models after base
    if i > 0 :
        estimator = make_model()
        gc.collect()
        estimator.set_weights(estimators_weights[i-1])
        gc.collect()
        estimator.compile(optimizer="RMSprop",loss=tf.keras.losses.BinaryCrossentropy(),metrics=['acc'])
        gc.collect()
        sample_weight = sample_weights[i-1]
        gc.collect()
    # base model
    else:
        estimator = make_model()
        gc.collect()
        estimator.compile(optimizer="RMSprop",loss=tf.keras.losses.BinaryCrossentropy(),metrics=['acc'])
        gc.collect()
        sample_weight = np.ones( x_train.shape[0] ) / x_train.shape[0]
        gc.collect()
    
        
    print(f"Estimator no. {i+1} \n\n")
    gc.collect()
    gc.collect()
    
    # base model and others could have different epochs
    if i > 0:
        estimator.fit(x_train, y_train, sample_weight = sample_weight, validation_data = [x_valid , y_valid], epochs = 1, batch_size = 32)
    else:
        estimator.fit(x_train, y_train, sample_weight = sample_weight, validation_data = [x_valid , y_valid], epochs = 1, batch_size = 32)
    gc.collect()
    gc.collect()
    
    # simple evaluation for each model
    print("\n\n")
    print(f"Train Acc: {estimator.evaluate(x_train, y_train)}\n\n")
    gc.collect()
    gc.collect()
    print(f"Test Acc: {estimator.evaluate(x_test, y_test)}\n\n")
    gc.collect()
    gc.collect()
    print(f"Valid Acc: {estimator.evaluate(x_valid, y_valid)}\n\n")
    gc.collect()
    gc.collect()
    
    
    # Step 2: Calculate the err equation
    #print("start step 2")
    predicted = (estimator.predict(x_train)).round()
    gc.collect()
    gc.collect()
    
    # 1 if the model classifiy it wrong, 0 then it is calssified it right
    incorrect = np.array([1 if i.all() == True else 0 for i in predicted != y_train ])
    del predicted
    gc.collect()
    
    error = np.dot(incorrect, sample_weight) / np.sum(sample_weight)
    #print(f"err = {error}\n\n")
    gc.collect()

    ############ we need to change this condition  ###############
    if error >= 1 - 1 / 2 and i > 0:
        print("stoped boosting")
        break
    
    #print("start step 3")
    # step 3: caluclate the alpha
    lr = K.eval(estimator.optimizer.lr)
    alpha = lr*(math.log((1-error)/error)) + math.log(2-1)
    del error, lr
    gc.collect()
    
    #print("start step 4")
    # step 4: update the weights
    sample_weight *= np.exp(alpha * incorrect) 
    gc.collect()
    sample_weight /= sample_weight.sum()
    
    # appending and cleaning before repeating
    estimators_alphas.append(alpha)
    del alpha , incorrect
    gc.collect()
    
    estimators.append(estimator)
    gc.collect()
    
    estimators_weights.append(estimator.get_weights())
    del estimator
    gc.collect()
    
    sample_weights.append(sample_weight)
    del sample_weight
    gc.collect()
    
    #print("finish \n")

del estimators_weights
gc.collect()

In [ ]:
print(f" Train Acc: {samme_evalution(x_train,y_train,n_e, estimators_alphas, estimators)}")
print(f" Test Acc: {samme_evalution(x_test,y_test,n_e, estimators_alphas, estimators)}")
print(f" Validation Acc: {samme_evalution(x_valid,y_valid,n_e, estimators_alphas, estimators)}")

# Notes


# SAMME.R

In [12]:
# Step 3: Calculate h
def Hk(predicted, k = 2):
    predicted = np.array(predicted)
    predicted[predicted < np.finfo(predicted.dtype).eps] = np.finfo(predicted.dtype).eps
    log_predict = np.log(predicted)
    h = (k-1) * (log_predict - 1/k * log_predict.sum())
    
    return h

def evaluate(x, y, estimators, n_e, hk = None, k = 2):
    if hk == None:
        hk = []
        for i in range(0,n_e):
            hk.append(estimators[i].predict(x))
            gc.collect()

        hk = np.array(hk)
        hk[hk < np.finfo(hk.dtype).eps] = np.finfo(hk.dtype).eps
        gc.collect()
        log_prop = np.log(hk)
        gc.collect()
        intermediate = (k-1) * (log_prop - 1/k * log_prop.sum())
        gc.collect()
        intermediate = intermediate.sum(axis = 0)
    else:
        intermediate = np.array(hk).sum(axis = 0)
    acc = 0
    for i in range(len(y)):
       if (np.argmax(intermediate[i])) == (np.argmax(y[i])):
            acc += 1 
    print(f"Accuracy : {acc/y.shape[0]}")
    return acc/y.shape[0]

In [59]:
gc.collect()
tf.random.set_seed(42)
gc.collect()

estimators = [] # list of models
estimators_weights = [] # list of CNN weights of each model
sample_weights = [] # list of sample weights of each model
#H = [] # list of h of each model

estimators_pred = []

y_labels = [ i[0] for i in y_train]
y_coding = np.array([1 if i == 1 else -1 for i in y_labels])

n_e = 10
gc.collect()

for i in range(0, n_e):
    gc.collect()
    # models after base
    if i > 0 :
        estimator = make_model() #MobileNet()
        gc.collect()
        estimator.set_weights(estimators_weights[i-1])
        gc.collect()
        estimator.compile(optimizer="adam",loss=tf.keras.losses.BinaryCrossentropy(),metrics=['acc'])
        gc.collect()
        sample_weight = sample_weights[i-1]
        gc.collect()
    
    # base model
    else:
        estimator = make_model() #MobileNet()
        gc.collect()
        estimator.compile(optimizer="RMSprop",loss=tf.keras.losses.BinaryCrossentropy(),metrics=['acc'])
        gc.collect()
        sample_weight = np.ones(x_train.shape[0]) / x_train.shape[0]
        gc.collect()
    
        
    print(f"Estimator no. {i+1} \n\n")
    gc.collect()
    gc.collect()
    
    # base model and others could have different epochs
    if i > 0:
        estimator.fit(x_train, y_train, sample_weight=sample_weight, validation_data = [x_valid , y_valid], epochs = 1, batch_size = 32)
    else:
        estimator.fit(x_train, y_train, sample_weight=sample_weight, validation_data = [x_valid , y_valid], epochs = 7, batch_size = 8)
    gc.collect()
    gc.collect()
    
    # simple evaluation for each model
    print("\n\n")
    print(f"Train Acc: {estimator.evaluate(x_train, y_train)}\n\n")
    gc.collect()
    gc.collect()
    print(f"Test Acc: {estimator.evaluate(x_test, y_test)}\n\n")
    gc.collect()
    gc.collect()
    print(f"Valid Acc: {estimator.evaluate(x_valid, y_valid)}\n\n")
    gc.collect()
    gc.collect()
    
    ############# first we need to calculate the alpha for early stopping ############
    #################################################################################
    ################### DONT, DONT, DONT FOOOOOORGEEEEEEET #########################
    ################### DONT, DONT, DONT FOOOOOORGEEEEEEET #########################
    ###############################################################################
    
    # Step 2: get class predicted prop
    #print("start step 2")
    predicted = (estimator.predict(x_train, verbose=0))
    gc.collect()
    gc.collect()
    
    prop = np.array([predicted[i][0] for i in range(len(y_train))])
    prop[prop < np.finfo(prop.dtype).eps] = np.finfo(prop.dtype).eps
    gc.collect()
    gc.collect()
    
    # Step 3: Calculate h
    #predicted = np.array(predicted)
    #H.append(Hk(predicted))
    #del predicted
    #gc.collect()
    #gc.collect()
    
    #print("start step 4")
    # step 4: update the weights
    lr =  1 #K.eval(estimator.optimizer.lr)
    intermediate_value = np.exp(-lr * (((2 - 1) / 2) * (np.abs(y_coding)* np.log(prop))))
    gc.collect()
    sample_weight = sample_weight * intermediate_value
    gc.collect()
    sample_weight = sample_weight / sample_weight.sum()
    del intermediate_value, prop, lr
    gc.collect()
    
    # appending and cleaning before repeating
    estimators.append(estimator)
    gc.collect()
    
    estimators_weights.append(estimator.get_weights())
    del estimator
    gc.collect()
    
    sample_weights.append(sample_weight)
    del sample_weight
    gc.collect()
    #print("finish \n")

del estimators_weights
gc.collect()

Estimator no. 1 


Epoch 1/7
281/281 [==============================] - 45s 129ms/step - loss: 3.0936e-04 - acc: 0.5024 - val_loss: 0.6575 - val_acc: 0.6800
Epoch 2/7
281/281 [==============================] - 35s 125ms/step - loss: 2.7073e-04 - acc: 0.6739 - val_loss: 1.1993 - val_acc: 0.7240
Epoch 3/7
281/281 [==============================] - 35s 125ms/step - loss: 2.1912e-04 - acc: 0.7817 - val_loss: 0.4499 - val_acc: 0.7960
Epoch 4/7
281/281 [==============================] - 35s 125ms/step - loss: 1.8204e-04 - acc: 0.8214 - val_loss: 0.4442 - val_acc: 0.8280
Epoch 5/7
281/281 [==============================] - 35s 125ms/step - loss: 1.5443e-04 - acc: 0.8588 - val_loss: 1.5483 - val_acc: 0.6880
Epoch 6/7
281/281 [==============================] - 35s 125ms/step - loss: 1.3206e-04 - acc: 0.8784 - val_loss: 1.1936 - val_acc: 0.7720
Epoch 7/7
281/281 [==============================] - 35s 125ms/step - loss: 1.0976e-04 - acc: 0.9078 - val_loss: 0.6550 - val_acc: 0.7800



71/71 [=====

0

In [60]:
print(f" Train Acc: {evaluate(x_train, y_train, estimators, n_e, k = 2)}")
print(f" Test Acc: {evaluate(x_test, y_test, estimators, n_e, k = 2)}")
print(f" Validation Acc: {evaluate(x_valid, y_valid, estimators, n_e, k = 2)}")

71/71 [==============================] - 5s 76ms/step
Accuracy : 0.9487750556792873
 Train Acc: 0.9487750556792873
14/14 [==============================] - 1s 77ms/step
Accuracy : 0.8163265306122449
 Test Acc: 0.8163265306122449
8/8 [==============================] - 1s 74ms/step
Accuracy : 0.824
 Validation Acc: 0.824


## Notes 